In [1]:
import sys


import numpy as np
import os
import torch
from diffusionLM.pipeline import diffusionLM 
import pandas as pd
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
from rdkit import Chem
from rdkit.Chem import Draw, AllChem, rdchem
from IPython.display import Image, display
import sys
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


/home/ec2-user/anaconda3/envs/diffusionLM_github/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/ec2-user/anaconda3/envs/diffusionLM_github/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [6]:
pipeline = diffusionLM()

# Data
data_config = pipeline.get_default_data_config()
data_config.task = "conditional"
data_config.trainset_path = sys.path[0] + "/diffusionLM/datasets/htpmd_conditional"
# model config
print(data_config.trainset_path)
model_config = pipeline.get_default_model_config()

# Generate train job file
# Please go to "improved-diffusion" folder and run "bash train_(un)conditional.sh"
train_config = pipeline.get_default_train_config()
pipeline.train_job(model_config, data_config, train_config)

# Generate sampling job file
# Once the training is done, please go to "improved-diffusion" folder and run "bash generation_(un)conditional.sh" to generate samples
generate_config = pipeline.get_default_generate_config()
generate_config.model_index = "010000" # ckpt used for generation
pipeline.generate_job(model_config, data_config, train_config, generate_config)

# Evaluation of the generation
# The generated results are stored under "improved-diffusion/generation_outputs"
generated_poly_file = sys.path[0] + "/diffusionLM/improved-diffusion/generation_outputs/conditional/generate_mol_epoch100000.txt"

print("########################")
print("Evaluation metrics: uniqueness, novelty, validity, synthesibility, similarity, diversity")
print(pipeline.evaluate(generated_poly_file, "conditional"))

/home/ec2-user/SageMaker/github/PolyGen-main/diffusionLM/datasets/htpmd_conditional
python /home/ec2-user/SageMaker/github/PolyGen-main/diffusionLM/improved-diffusion/scripts/run_train.py --diff_steps 2000 --model_arch transformer --lr 0.0001 --lr_anneal_steps 100000 --seed 101 --noise_schedule sqrt --in_channel 16 --padding_mode pad --bsz 64 --modality roc --submit no --notes xstart_e2e --app "--predict_xstart True --training_mode e2e  --vocab_size 19 --roc_train /home/ec2-user/SageMaker/github/PolyGen-main/diffusionLM/datasets/htpmd_conditional"
########################
Evaluation metrics: uniqueness, novelty, validity, synthesibility, similarity, diversity
(0.6763754045307444, 0.13915857605177995, 0.9870550161812298, 0.9736842105263158, 0.29085734336792707, 0.6869902464627571)
